# Getting started with uploading your data to the atlas website

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/int-brain-lab/iblbrainviewer/blob/main/docs/atlas_website_getting_started.ipynb)

The website [http://atlas.internationalbrainlab.org](http://atlas.internationalbrainlab.org) enables you to upload and region, volumetric and point data and explore it and in interactive way using different anatomical views.

This notebook walks you through how you can prepare some region based data and shows you how can upload it onto the website.


## 1. Install the necessary packages on your machine

In [ ]:
!pip install iblbrainviewer

## 2. Prepare your data

In this example we show how to upload region based data. For this type of data you will have to declare:

- A `regions` array containing the brain regions to be displayed. The regions can be given either as Allen `acronyms` or `atlas_ids`
- One (or more) `feature` array(s) containing the values to be displayed, each corresponding to a brain region listed in the `regions` array
- The names of each features


The list of possible brain regions contained in the Allen structure tree can be explored in the following way

In [1]:
from iblatlas.regions import BrainRegions
import numpy as np

br = BrainRegions()

acronyms = np.unique(br.acronym)

# Print the list of brain regions (first 100)
print(f'There are {len(acronyms)} acronyms; first 10 are named: {acronyms[0:10]} ...')

There are 1328 acronyms; first 10 are named: ['6b' 'A13' 'AAA' 'ACA' 'ACA1' 'ACA2/3' 'ACA5' 'ACA6a' 'ACA6b' 'ACAd'] ...


Let's imagine you want to plot 3 features (e.g. the mean spike rate, amplitude, or LFP phase), for the following 4 brain regions : MOp1, MOs5, AAA and ACAd2/3. The arrays and associated variables would be:

In [2]:
acronyms = np.array(['MOp1', 'MOs5', 'AAA', 'ACAd2/3'])

# Spike rate features
fname1 = 'spike_rate'
values1 = np.array([1.2, 3.4, 0.0, 0.1])

# Spike amplitude feature
fname2 = 'spike_amplitude'
values2 = np.array([33.5, 234.2, 5.3, 10.6])

# LFP phase feature
fname3 = 'lfp_phase'
values3 = np.array([36.0, 12.3, 34.0, 56.0])

It is important that the regions and features are the same size

In [3]:
# Check that the arrays have the same length
assert acronyms.size == values1.size
assert acronyms.size == values2.size
assert acronyms.size == values3.size

## 3. Understand some useful concepts

Before we upload our example data to the website, we will introduce you to some useful concpets that underly how the data you upload is structured and organised.


### Bucket
All data are organised into buckets. A bucket acts like a folder and contains all data common to a given project or user. A single bucket can contain multiple features.

### Local Bucket
A local bucket is a folder on your local filesystem. Features can be added to the local bucket and uploaded to the website. The data is stored in the browser cache and will persist until the browser cache is cleared. All users have to their own unique local bucket and the data can only be accessed by the local user.

### Remote Bucket
A remote bucket is a folder that can be created on a remote server. When adding data to a remote bucket, the data is uploaded to our server and then shown on the website. This data will always be available (see storage terms and conditions) and can be viewed by multiple users.

### Local vs remote bucket
The main difference between data stored on a local vs a remote bucket is the ability for the data to persist and to be shared with others. 

### Features
A feature is the data to be displayed. A feature can be 

- Region based - a map of individual brain regions to values (e.g `spike_rate` or  `spike_amplitude` defined above)
- Volume based - full 3D volume
- Point based - 3D coordinates of individual points (e.g clusters or channels)

A bucket can contain many features. The name of each feature within a bucket must be unique

### Tree
A tree defines the organistion of features within a bucket. Features can be arranged into folders within folders or renamed for display purposes. Trees can only be applied to remote buckets.


## 3. Upload your data to the website via a local bucket

The feature uploader API is used to upload your data to the `atlas` website. To start uploading your data to the local bucket, import the feature uploader and instantiate it. 

In [4]:
from iblbrainviewer.api import FeatureUploader

up = FeatureUploader()

Notice how the bucket name is `local`

In [5]:
print(up.bucket_uuid)

local


We need to define a path on our local computer that will store our local data

In [6]:
from pathlib import Path
bucket_path = Path.home().joinpath('local_bucket')
bucket_path.mkdir(exist_ok=True)
print(bucket_path)

/Users/admin/local_bucket


Now we can upload our region based features to our bucket

In [7]:
up.local_features(fname1, acronyms, values1, hemisphere='left', output_dir=bucket_path)
up.local_features(fname2, acronyms, values2, hemisphere='left', output_dir=bucket_path)
up.local_features(fname3, acronyms, values3, hemisphere='left', output_dir=bucket_path)

If you now navigate to the location of `bucket_path` on your local file system you should see the three features. 
 - lfp_phase.json
 - spike_amplitude.json
 - spike_rate.json

To upload these features to the website, select the local bucket from the drop down menu on the top left hand side of the webpage and click on the up arrow button and choose a feature or multiple features to upload.

## 4. Upload your data to the website via a remote bucket

To upload your data to a remote bucket you will first need to define bucket name.  The name of the bucket should ideally easily identify what your bucket will contain, or who it belongs to. You can follow the convention `my3initials__myproject`, e.g. `sebcha_serotonin`. We can then import the FeatureUploader and this time instantiate it with the bucket name.

In [8]:
from iblbrainviewer.api import FeatureUploader

bucket = 'ibl_getting_started'
# Create or load the bucket
up = FeatureUploader(bucket)

The first time you create a bucket you will be prompted to enter a global key. The global key is `f9134149-fcb5-4142-a0eb-4e76a0811cf9`. Once the FeatureUploader is instantiated you can upload your features to the bucket

In [9]:
# Create the features.
if not up.features_exist(fname1):
    up.create_features(fname1, acronyms, values1, hemisphere='left')
if not up.features_exist(fname2):
    up.create_features(fname2, acronyms, values2, hemisphere='left')
if not up.features_exist(fname3):
    up.create_features(fname3, acronyms, values3, hemisphere='left')

Once you have uploaded your features you can then generate a url to your bucket in the following way

In [10]:
url = up.get_buckets_url([bucket])
print(url)

https://atlas.internationalbrainlab.org/?buckets=ibl_getting_started&bucket=ibl_getting_started


If you follow this link you should see these example features in the dropdown tree

### Defining a tree
A tree can be added to a remote bucket to define the organistaion of the features in your bucket. By default, when a bucket is created it does not contain a tree. On the display all the features will be named by their variable names `fname`, and on the same hierarchy level (i.e. as one drop-down list).

If you want to organise your features into groups, or change the naming of the features displayed you can define a tree. 

The tree allows you to
- Hierachically organise the features in your bucket
- Change the name displayed for each feature on the webpage
- Exclude the display of chosen features

For example here, we will collate the spikes feature into a group called "Spike" and also rename the feature names

In [11]:
tree = {'Spike': {'mean spike rate (Hz)': fname1,
                  'mean spike amplitude (uV)': fname2},
        'LFP phase (8-10Hz band) (rad)': fname3}

We can apply the tree to the bucket in the following way

In [12]:
up.patch_bucket(tree=tree)
url = up.get_buckets_url([bucket])
print(url)

https://atlas.internationalbrainlab.org/?buckets=ibl_getting_started&bucket=ibl_getting_started


If you open the url again and hit the refresh button that you see that at the top centre of the box showing your features list, you should find that the features have been rearranaged

## 5. Sharing your data with others

The url generated above can be shared with others and they will be able to view the data in your remote bucket. Alternatively, if the unique name of the remote bucket is known, the `+` button on the left hand side of the website can be clicked, the name entered and the bucket loaded. 

A unique token is automatically generated upon creation of a new remote bucket and stored locally on the users computer. This ensures that while the remote bucket can viewed by anyone with the url or the bucket name, only people with access to this unique token can upload, change or delete data contained within the remote bucket. Please refer to the **Instantiating a bucket with a token** section in the [API tutorial](https://github.com/int-brain-lab/iblbrainviewer/blob/main/docs/atlas_website_using_the_API.ipynb) for an example of how to share the bucket token between users.

## 5. Further Reading

- [Read more preparing and uploading region data to the website](https://github.com/int-brain-lab/iblbrainviewer/blob/main/docs/atlas_website_preparing_region_data.ipynb)
- [Read more preparing and uploading volumetric data to the website](https://github.com/int-brain-lab/iblbrainviewer/blob/main/docs/atlas_website_preparing_volumetric_data.ipynb)
- [Read more preparing and uploading point data to the website](https://github.com/int-brain-lab/iblbrainviewer/blob/main/docs/atlas_website_preparing_point_data.ipynb)
- [Read more about the different mappings used to display the data](https://github.com/int-brain-lab/iblbrainviewer/blob/main/docs/atlas_website_understanding_the_mappings.ipynb)
- [Get familiar with the FeatureUploader API](https://github.com/int-brain-lab/iblbrainviewer/blob/main/docs/atlas_website_using_the_API.ipynb)
- [Get familiar with using the website](https://github.com/int-brain-lab/iblbrainviewer/blob/main/docs/atlas_website_using_the_website.ipynb)